In [1]:
import pandas as pd
import requests
import json
import datetime as dt
import os

# Generation Data

In [2]:
json_file_path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/generation_data/'

os.listdir(json_file_path)

['loanda_data_curated_0.json',
 'loanda_data_curated_200000.json',
 'loanda_data_curated_400000.json',
 'loanda_data_curated_600000.json',
 'loanda_data_curated_800000.json']

In [3]:
json_file_path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/generation_data/'
df_generation = pd.DataFrame()
for data_chunks in os.listdir(json_file_path):
    path = json_file_path + data_chunks
    with open(path, "r") as json_file:
    # Load the JSON data into a Python object.
        json_data = json.load(json_file)
        pd_json_data = pd.DataFrame.from_dict(json_data)
        df_generation = pd.concat([df_generation,pd_json_data])

In [4]:
df_generation

,E3TimeStamp,PotenciaAtivaTotal,EnergiaTotal_DataLogger,EnergiaDiaria_DataLogger,RadiacaoGlobal,TempAr,TempModulo,VelocidadeVento,EnergiaTotal_LoteB,EnergiaDiaria_LoteB,...,EnergiaReativa_LoteC,EnergiaTotal_LoteD,EnergiaDiaria_LoteD,EnergiaReativa_LoteD,EnergiaTotal_LoteE,EnergiaDiaria_LoteE,EnergiaReativa_LoteE,EnergiaTotal_LoteF,EnergiaDiaria_LoteF,EnergiaReativa_LoteF
0,2021-08-17 07:42:20.090000000,288,413620,51,103.0,21.0,-10.0,1.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2021-08-17 07:43:20.043000000,297,413630,56,107.0,21.0,-10.0,2.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2021-08-17 07:44:20.037000000,305,413630,61,111.0,21.0,-10.0,4.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2021-08-17 07:45:20.127000000,316,413640,66,115.0,21.0,-10.0,3.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2021-08-17 07:46:20.027000000,326,413640,71,118.0,21.0,-10.0,5.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949868,2023-08-09 22:16:10.357000000,0,3376782,5634,-3.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0
949869,2023-08-09 22:17:10.407000000,0,3376782,5634,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0
949870,2023-08-09 22:18:10.380000000,0,0,0,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0
949871,2023-08-09 22:19:10.373000000,0,0,0,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0


# Weather Data

In [6]:
path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/weather_data/loanda_weather_curated.json'
with open(path, "r") as json_file:
# Load the JSON data into a Python object.
    json_data = json.load(json_file)
    df_weather = pd.DataFrame.from_dict(json_data)

# Concat weather e generation data

In [8]:
df_generation['E3TimeStamp'] = pd.to_datetime(df_generation['E3TimeStamp'], format='mixed')
df_generation['time_join_generation'] = df_generation['E3TimeStamp'].dt.date.astype(str) +' '+ df_generation['E3TimeStamp'].dt.hour.astype(str)
df_weather['time_in_iso8601'] = pd.to_datetime(df_weather['time_in_iso8601'], format='ISO8601')
df_weather['time_join_generation'] = df_weather['time_in_iso8601'].dt.date.astype(str) +' '+ df_weather['time_in_iso8601'].dt.hour.astype(str)

In [12]:
df = pd.merge(df_generation, df_weather, how='left', on='time_join_generation')

# EDA

In [9]:
df_generation

,E3TimeStamp,PotenciaAtivaTotal,EnergiaTotal_DataLogger,EnergiaDiaria_DataLogger,RadiacaoGlobal,TempAr,TempModulo,VelocidadeVento,EnergiaTotal_LoteB,EnergiaDiaria_LoteB,...,EnergiaTotal_LoteD,EnergiaDiaria_LoteD,EnergiaReativa_LoteD,EnergiaTotal_LoteE,EnergiaDiaria_LoteE,EnergiaReativa_LoteE,EnergiaTotal_LoteF,EnergiaDiaria_LoteF,EnergiaReativa_LoteF,time_join_generation
0,2021-08-17 07:42:20.090,288,413620,51,103.0,21.0,-10.0,1.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2021-08-17 7
1,2021-08-17 07:43:20.043,297,413630,56,107.0,21.0,-10.0,2.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2021-08-17 7
2,2021-08-17 07:44:20.037,305,413630,61,111.0,21.0,-10.0,4.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2021-08-17 7
3,2021-08-17 07:45:20.127,316,413640,66,115.0,21.0,-10.0,3.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2021-08-17 7
4,2021-08-17 07:46:20.027,326,413640,71,118.0,21.0,-10.0,5.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,2021-08-17 7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949868,2023-08-09 22:16:10.357,0,3376782,5634,-3.0,21.0,19.0,3.0,4325685.6,5570.4,...,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0,2023-08-09 22
949869,2023-08-09 22:17:10.407,0,3376782,5634,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0,2023-08-09 22
949870,2023-08-09 22:18:10.380,0,0,0,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0,2023-08-09 22
949871,2023-08-09 22:19:10.373,0,0,0,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0,2023-08-09 22
